### source data set : urban sound
* https://github.com/rickiepark/tfk-notebooks/tree/master/urban-sound-classification
* sound_data = np.load('urban_sound.npz')
* 뉴욕 대학교 MARL에서 2014년 공개한 데이터

### 1. data set preprocessing
* 원본 데이터셋은 class가 뒤죽박죽 섞여 있었음.
* 이를 class별로 정리하기 위한 전처리
    * air_conditioner, car_horn, children_playing, dog_bark, drilling
    * engine_idllling, gun_shot, jack_hammer, siren, street_music

---

* 이 때 각 wav file의 실행 시간이 다르면 학습에 문제가 생긴다. 실행 시간 통일이 필요하다.
    + 방법 1. 그냥 3.5초 미만의 모든 wav file은 복사하지 않는 방식 적용

    + 방법 2. 총소리는 4초만 뽑으면 16개 남는다.. 낮은 시간들도 붙여서 사용할 필요가 있다
        + 0.5초 이하의 wavfile은 가능하면 공백을 해당 길이만큼 두고, 4초로 맞춰서 붙이기
        + 2초 이하의 wav file은 붙여서 4초로 만들기
        + 2초 이상 4초 이하의 wav file은 
        + 4초 이상의 wav file은 4초 단위로 split하고 삭제하기

---

* 지금은 먼저 재생시간이 해당 범위에 포함되지 않으면 아예 분류하지 않고, 추후에 부가적인 작업으로 데이터셋 개수가 부족한 클래스에 대해 보충 작업을 하는 식으로 구성되어있다.

In [ ]:
import glob
import shutil
import wave
import time
import contextlib
import soundfile as sf
import numpy as np
from tqdm import tqdm_notebook

arr = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Name = ["air_conditioner", "car_horn", "children_playing", "dog_bark",
       "drilling", "engine_idlling", "gun_shot", "jackhammer", "siren", "street_music"]

def name(num):
    arr[num] = arr[num] + 1
    return Name[num] + str(arr[num])

In [ ]:
for i in range(1, 11):
    folderName = "fold"+str(i)
    fileList = glob.glob("1_origindata/"+folderName+"/*.wav")
    for j in tqdm_notebook(range(len(fileList))):
        originfileName = fileList[j]
        
        # 재생 시간을 구하는 코드
        data, fs = sf.read(originfileName)
        times = np.arange(len(data))/float(fs)
        duration = round(times[-1], 2)        
        
        # originfileName의 실행 시간을 측정해서 3.8초에서 4.2초 사이인 경우 복사 코드 수행.
        if(duration >= 3.8 and duration <= 4.2):
            tempSplit = originfileName.split("-")
            newfileName = "2_data/fold"+str(tempSplit[1])+"/"+name(int(tempSplit[1]))+".wav"
            shutil.copy(originfileName, newfileName)
            print("Success copy", end = " : ")
            print(originfileName + " -> " + newfileName)
                
        # 아닌 경우 originfileName 이름, 4초보다 짧은 실행 시간을 출력하며 복사하지 않음을 명시한다.
        else:
            print(originfileName + " can't copy.. play time is " + str(duration))
    print("/// " + folderName + " finish..")
print("all finish...")

In [ ]:
"""
* wav file이 24비트인 경우를 지원하지 않는 라이브러리가 많다.
* soundfile는 24비트 wav file을 지원하는 흔치 않는 경우
* BSD-3-Clause License
* https://github.com/bastibe/SoundFile
* 본 코드는 24비트 재생시간을 추출하기 위한 코드
"""
import soundfile as sf
import numpy as np
fname = "1_origindata/fold1/103074-7-0-0.wav"
fname = "1_origindata/fold1/102842-3-1-6.wav"

data, fs = sf.read(fname)
times = np.arange(len(data))/float(fs)
duration = round(times[-1], 2)

In [4]:
""" 
* gun shot sound 4초 단위 파일로 재생성하기 위한 코드
* 다른 데이터는 개수가 충분하기 때문에 무관하다.
* 경적소리가 209개, 총소리 16개로 너무 적다.
* 가능한 최대로 데이터로 재생성할 필요는 존재한다.
"""
import glob
from tqdm import tqdm_notebook
from pydub import AudioSegment
import soundfile as sf
import numpy as np
import shutil

className = "gun_shot"

fileList = glob.glob(className + "/*.wav")
for j in tqdm_notebook(range(len(fileList))):
    originfileName = fileList[j]
        
    # 재생 시간을 구하는 코드
    data, fs = sf.read(originfileName)
    times = np.arange(len(data))/float(fs)
    duration = round(times[-1], 2)        
    """
    # 2초 이하의 경우 4초/duration만큼 반복해서 붙이기
    if(duration <= 2.0):
        sound2 = AudioSegment.from_wav(originfileName)
        for t in range(int(4.0 / duration)):
            sound1 = AudioSegment.from_wav(originfileName)
            combined_sounds = sound1 + sound2
            combined_sounds.export(originfileName, format="wav")      
    """
    # 2초 이상 3.8초 이하의 경우 2초를 기준으로 나눠서 앞은 2배하고, 뒤는 4초/duration만큼 반복해서 붙이기
    # 단, 지금으로서는 일단 그냥 스킵. 대신 2.0초~2.2초 정도는 *2배 해서 저장하는것으로 대체
    if(duration <= 2.5):
        try:
            sound1 = AudioSegment.from_wav(originfileName)
            sound2 = AudioSegment.from_wav(originfileName)
            combined_sounds = sound1 + sound2
            combined_sounds.export(originfileName, format="wav")
        except:
            print(originfileName + " have path error")
            continue
            
    # originfileName의 실행 시간을 측정해서 3.8초에서 4.2초 사이인 경우 복사 코드 수행.
    elif(duration > 4.2):
        continue
        
    newfileName = className + "_change/"+className+str(j)+".wav"
    shutil.copy(originfileName, newfileName)
    print("Success copy", end = " : ")
    print(originfileName + " -> " + newfileName)

C:\ProgramData\Anaconda3\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Success copy : gun_shot\gun_shot10.wav -> gun_shot_change/gun_shot0.wav
Success copy : gun_shot\gun_shot100.wav -> gun_shot_change/gun_shot1.wav
Success copy : gun_shot\gun_shot107.wav -> gun_shot_change/gun_shot8.wav
Success copy : gun_shot\gun_shot108.wav -> gun_shot_change/gun_shot9.wav
Success copy : gun_shot\gun_shot109.wav -> gun_shot_change/gun_shot10.wav
Success copy : gun_shot\gun_shot11.wav -> gun_shot_change/gun_shot11.wav
Success copy : gun_shot\gun_shot113.wav -> gun_shot_change/gun_shot15.wav
Success copy : gun_shot\gun_shot114.wav -> gun_shot_change/gun_shot16.wav
Success copy : gun_shot\gun_shot115.wav -> gun_shot_change/gun_shot17.wav
Success copy : gun_shot\gun_shot116.wav -> gun_shot_change/gun_shot18.wav
Success copy : gun_shot\gun_shot117.wav -> gun_shot_change/gun_shot19.wav
Success copy : gun_shot\gun_shot118.wav -> gun_shot_change/gun_shot20.wav
Success copy : gun_shot\gun_shot119.wav -> gun_shot_change/gun_shot21.wav
Success copy : gun_shot\gun_shot12.wav -> gu

Success copy : gun_shot\gun_shot363.wav -> gun_shot_change/gun_shot291.wav
Success copy : gun_shot\gun_shot364.wav -> gun_shot_change/gun_shot292.wav
Success copy : gun_shot\gun_shot365.wav -> gun_shot_change/gun_shot293.wav
Success copy : gun_shot\gun_shot366.wav -> gun_shot_change/gun_shot294.wav
Success copy : gun_shot\gun_shot376.wav -> gun_shot_change/gun_shot305.wav
Success copy : gun_shot\gun_shot38.wav -> gun_shot_change/gun_shot306.wav
Success copy : gun_shot\gun_shot39.wav -> gun_shot_change/gun_shot307.wav
Success copy : gun_shot\gun_shot4.wav -> gun_shot_change/gun_shot308.wav
Success copy : gun_shot\gun_shot42.wav -> gun_shot_change/gun_shot311.wav
Success copy : gun_shot\gun_shot44.wav -> gun_shot_change/gun_shot313.wav
Success copy : gun_shot\gun_shot45.wav -> gun_shot_change/gun_shot314.wav
Success copy : gun_shot\gun_shot46.wav -> gun_shot_change/gun_shot315.wav
Success copy : gun_shot\gun_shot47.wav -> gun_shot_change/gun_shot316.wav
Success copy : gun_shot\gun_shot48

In [ ]:
""" 
* gun shot sound 4초 단위 파일로 재생성하기 위한 코드
* 다른 데이터는 개수가 충분하기 때문에 무관하다.
* 경적소리가 209개, 총소리 16개로 너무 적다.
* 가능한 최대로 데이터로 재생성할 필요는 존재한다.
"""

# 2초 이하의 경우 4초/duration만큼 반복해서 붙이기
# 2초 이상 3.8초 이하의 경우 2초를 기준으로 나눠서 앞은 2배하고, 뒤는 4초/duration만큼 반복해서 붙이기    
# originfileName의 실행 시간을 측정해서 3.8초에서 4.2초 사이인 경우 복사 코드 수행.

"""
* pydyb : wav 파일을 합치거나 split할 때 쓰는 라이브러리
* https://github.com/jiaaro/pydub/blob/master/LICENSE
* MIT License
"""

from pydub import AudioSegment

wav1 = 'car_horn\car_horn143.wav'
wav2 = 'car_horn\car_horn143.wav'
wav3 = 'ccar_horn_change\car_horn143.wav'

sound1 = AudioSegment.from_wav(wav1)
sound2 = AudioSegment.from_wav(wav2)

combined_sounds = sound1 + sound2
combined_sounds.export(wav3, format="wav")

---

### 2. convert wav to jpg
* 1.librosa 라이브러리르 이용하여 wav파일을 jpg파일로 변환
     + 1, 3, 4, 6, 8, {0, 2, 5, 7, 9} class로 classifier 개발 시작.
     + 즉, 10개의 class가 있지만, 0, 2, 5, 7, 9로 판별되는 경우 백색소음으로 뺀다.

---
     
* 2.jpg로 변환할 때, 필요한 부분만 잘라서 저장한다.
    + 자르는 범위는 다음과 같다 area = (54, 32, 385, 253)

In [2]:
import librosa
import librosa.display
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import glob
from PIL import Image
from tqdm import tqdm_notebook

%matplotlib inline

In [5]:
"""1, 3, 4, 6, 8은 주인 찾아서, 나머지는 fold5에 몰아넣기"""
folderNameGateway = [5, 0, 6, 1, 2, 7, 3, 8, 4, 9]

for i in range(10):
    i = 6
    inputFolderName = "fold" + str(i)
    outputFolderName = "fold" + str(folderNameGateway[i])
    print("======"+inputFolderName+"======")
    fileList = glob.glob("2_data/"+inputFolderName+"/*.wav")
    
    """2_data에 있는 각 class에 대해 원하는 만큼 자르고 저장하기"""
    for j in tqdm_notebook(range(len(fileList))):
        tempSplit = fileList[j].split(".")
        tempSplit2 = tempSplit[0].split("\\")
        fileName = tempSplit2[-1]
        
        """wav to jpg"""
        plt.figure()
        y, sr = librosa.load(fileList[j])
        S = np.abs(librosa.stft(y))
        librosa.display.specshow(librosa.power_to_db(S**2, ref = np.max),
                                 sr = sr, y_axis = 'log', x_axis = 'time')
        #plt.colorbar(format = '%+2.0f dB')
        plt.title('Log-Power spectrogram')
        #plt.show()
        
        """임시 저장"""
        print("Save file Complete : 3_image/"+outputFolderName+"/"+fileName+".jpg")       
        plt.savefig("3_image/"+outputFolderName+"/"+fileName+".jpg")

        """임시 저장한 파일을 범위만큼 잘라서 다시 저장"""
        img = Image.open("3_image/"+outputFolderName+"/"+fileName+".jpg")
        area = (54, 32, 385, 253)
        cropped_img = img.crop(area)
        cropped_img.save("3_image/"+outputFolderName+"/"+fileName+".jpg")
        
        plt.close()
    break

======fold6======


Save file Complete : 3_image/fold3/gun_shot0.jpg
Save file Complete : 3_image/fold3/gun_shot1.jpg
Save file Complete : 3_image/fold3/gun_shot10.jpg
Save file Complete : 3_image/fold3/gun_shot101.jpg
Save file Complete : 3_image/fold3/gun_shot102.jpg
Save file Complete : 3_image/fold3/gun_shot105.jpg
Save file Complete : 3_image/fold3/gun_shot11.jpg
Save file Complete : 3_image/fold3/gun_shot120.jpg
Save file Complete : 3_image/fold3/gun_shot126.jpg
Save file Complete : 3_image/fold3/gun_shot130.jpg
Save file Complete : 3_image/fold3/gun_shot131.jpg
Save file Complete : 3_image/fold3/gun_shot132.jpg
Save file Complete : 3_image/fold3/gun_shot133.jpg
Save file Complete : 3_image/fold3/gun_shot134.jpg
Save file Complete : 3_image/fold3/gun_shot135.jpg
Save file Complete : 3_image/fold3/gun_shot137.jpg
Save file Complete : 3_image/fold3/gun_shot139.jpg
Save file Complete : 3_image/fold3/gun_shot141.jpg
Save file Complete : 3_image/fold3/gun_shot144.jpg
Save file Complete : 3_image/fold3/gu

In [ ]:
# 시각화 origin code backup
"""
plt.figure()
y, sr = librosa.load(fileList[0])
S = np.abs(librosa.stft(y))
librosa.display.specshow(librosa.power_to_db(S**2, ref = np.max),
                         sr = sr, y_axis = 'log', x_axis = 'time')
plt.colorbar(format = '%+2.0f dB')
plt.title('Log-Power spectrogram')
plt.show()
"""

---

### 3. split dataset
* 통합 database를 train, validation, test로 분할
* 전체 데이터 10이라 할때, train(6), val(2), test(2)

In [6]:
import glob
import shutil

for i in range(10):
    folderName = "fold" + str(i)
    fileList = glob.glob("3_image/"+folderName+"/*.jpg")
    trainRange = int(len(fileList)*0.6)
    validationRange = int(len(fileList)*0.2)
    
    print("train : 0:"+str(trainRange))
    print("validation : "+str(trainRange+1)+":"+str(trainRange+validationRange))
    print("test : "+str(trainRange+validationRange+1)+":"+str(len(fileList)))
    
    # train data
    for j in range(trainRange+1):
        originfileName = fileList[j]
        tempSplit = fileList[j].split("/")
        fileName = tempSplit[-1]
        
        newfileName = "4_dataset/1_train/"+fileName
        shutil.copy(originfileName, newfileName)
        print("copy succes : " + originfileName + " -> " + newfileName)

    # validation data
    for j in range(trainRange+1, trainRange + validationRange + 1):
        originfileName = fileList[j]
        tempSplit = fileList[j].split("/")
        fileName = tempSplit[-1]        
        
        newfileName = "4_dataset/2_val/"+fileName
        shutil.copy(originfileName, newfileName)
        print("copy succes : " + originfileName + " -> " + newfileName)
        
    # test data
    for j in range(trainRange + validationRange + 1, len(fileList)):
        originfileName = fileList[j]
        tempSplit = fileList[j].split("/")
        fileName = tempSplit[-1]        
        
        newfileName = "4_dataset/3_test/"+fileName
        shutil.copy(originfileName, newfileName)
        print("copy succes : " + originfileName + " -> " + newfileName)        

train : 0:257
validation : 258:342
test : 343:429
copy succes : 3_image/fold0\car_horn1.jpg -> 4_dataset/1_train/fold0\car_horn1.jpg
copy succes : 3_image/fold0\car_horn10.jpg -> 4_dataset/1_train/fold0\car_horn10.jpg
copy succes : 3_image/fold0\car_horn100.jpg -> 4_dataset/1_train/fold0\car_horn100.jpg
copy succes : 3_image/fold0\car_horn101.jpg -> 4_dataset/1_train/fold0\car_horn101.jpg
copy succes : 3_image/fold0\car_horn102.jpg -> 4_dataset/1_train/fold0\car_horn102.jpg
copy succes : 3_image/fold0\car_horn103.jpg -> 4_dataset/1_train/fold0\car_horn103.jpg
copy succes : 3_image/fold0\car_horn104.jpg -> 4_dataset/1_train/fold0\car_horn104.jpg
copy succes : 3_image/fold0\car_horn105.jpg -> 4_dataset/1_train/fold0\car_horn105.jpg
copy succes : 3_image/fold0\car_horn106.jpg -> 4_dataset/1_train/fold0\car_horn106.jpg
copy succes : 3_image/fold0\car_horn107.jpg -> 4_dataset/1_train/fold0\car_horn107.jpg
copy succes : 3_image/fold0\car_horn108.jpg -> 4_dataset/1_train/fold0\car_horn108.jpg

copy succes : 3_image/fold0\car_horn183.jpg -> 4_dataset/1_train/fold0\car_horn183.jpg
copy succes : 3_image/fold0\car_horn184.jpg -> 4_dataset/1_train/fold0\car_horn184.jpg
copy succes : 3_image/fold0\car_horn185.jpg -> 4_dataset/1_train/fold0\car_horn185.jpg
copy succes : 3_image/fold0\car_horn186.jpg -> 4_dataset/1_train/fold0\car_horn186.jpg
copy succes : 3_image/fold0\car_horn187.jpg -> 4_dataset/1_train/fold0\car_horn187.jpg
copy succes : 3_image/fold0\car_horn188.jpg -> 4_dataset/1_train/fold0\car_horn188.jpg
copy succes : 3_image/fold0\car_horn189.jpg -> 4_dataset/1_train/fold0\car_horn189.jpg
copy succes : 3_image/fold0\car_horn19.jpg -> 4_dataset/1_train/fold0\car_horn19.jpg
copy succes : 3_image/fold0\car_horn190.jpg -> 4_dataset/1_train/fold0\car_horn190.jpg
copy succes : 3_image/fold0\car_horn191.jpg -> 4_dataset/1_train/fold0\car_horn191.jpg
copy succes : 3_image/fold0\car_horn192.jpg -> 4_dataset/1_train/fold0\car_horn192.jpg
copy succes : 3_image/fold0\car_horn193.jpg -

copy succes : 3_image/fold0\car_horn268.jpg -> 4_dataset/1_train/fold0\car_horn268.jpg
copy succes : 3_image/fold0\car_horn269.jpg -> 4_dataset/1_train/fold0\car_horn269.jpg
copy succes : 3_image/fold0\car_horn27.jpg -> 4_dataset/1_train/fold0\car_horn27.jpg
copy succes : 3_image/fold0\car_horn270.jpg -> 4_dataset/1_train/fold0\car_horn270.jpg
copy succes : 3_image/fold0\car_horn271.jpg -> 4_dataset/1_train/fold0\car_horn271.jpg
copy succes : 3_image/fold0\car_horn272.jpg -> 4_dataset/1_train/fold0\car_horn272.jpg
copy succes : 3_image/fold0\car_horn273.jpg -> 4_dataset/1_train/fold0\car_horn273.jpg
copy succes : 3_image/fold0\car_horn274.jpg -> 4_dataset/1_train/fold0\car_horn274.jpg
copy succes : 3_image/fold0\car_horn275.jpg -> 4_dataset/1_train/fold0\car_horn275.jpg
copy succes : 3_image/fold0\car_horn276.jpg -> 4_dataset/1_train/fold0\car_horn276.jpg
copy succes : 3_image/fold0\car_horn277.jpg -> 4_dataset/1_train/fold0\car_horn277.jpg
copy succes : 3_image/fold0\car_horn278.jpg -

copy succes : 3_image/fold0\car_horn352.jpg -> 4_dataset/2_val/fold0\car_horn352.jpg
copy succes : 3_image/fold0\car_horn353.jpg -> 4_dataset/2_val/fold0\car_horn353.jpg
copy succes : 3_image/fold0\car_horn354.jpg -> 4_dataset/2_val/fold0\car_horn354.jpg
copy succes : 3_image/fold0\car_horn355.jpg -> 4_dataset/2_val/fold0\car_horn355.jpg
copy succes : 3_image/fold0\car_horn356.jpg -> 4_dataset/2_val/fold0\car_horn356.jpg
copy succes : 3_image/fold0\car_horn357.jpg -> 4_dataset/2_val/fold0\car_horn357.jpg
copy succes : 3_image/fold0\car_horn358.jpg -> 4_dataset/2_val/fold0\car_horn358.jpg
copy succes : 3_image/fold0\car_horn359.jpg -> 4_dataset/2_val/fold0\car_horn359.jpg
copy succes : 3_image/fold0\car_horn36.jpg -> 4_dataset/2_val/fold0\car_horn36.jpg
copy succes : 3_image/fold0\car_horn360.jpg -> 4_dataset/2_val/fold0\car_horn360.jpg
copy succes : 3_image/fold0\car_horn361.jpg -> 4_dataset/2_val/fold0\car_horn361.jpg
copy succes : 3_image/fold0\car_horn362.jpg -> 4_dataset/2_val/fold

copy succes : 3_image/fold0\car_horn52.jpg -> 4_dataset/3_test/fold0\car_horn52.jpg
copy succes : 3_image/fold0\car_horn53.jpg -> 4_dataset/3_test/fold0\car_horn53.jpg
copy succes : 3_image/fold0\car_horn54.jpg -> 4_dataset/3_test/fold0\car_horn54.jpg
copy succes : 3_image/fold0\car_horn55.jpg -> 4_dataset/3_test/fold0\car_horn55.jpg
copy succes : 3_image/fold0\car_horn56.jpg -> 4_dataset/3_test/fold0\car_horn56.jpg
copy succes : 3_image/fold0\car_horn57.jpg -> 4_dataset/3_test/fold0\car_horn57.jpg
copy succes : 3_image/fold0\car_horn58.jpg -> 4_dataset/3_test/fold0\car_horn58.jpg
copy succes : 3_image/fold0\car_horn59.jpg -> 4_dataset/3_test/fold0\car_horn59.jpg
copy succes : 3_image/fold0\car_horn6.jpg -> 4_dataset/3_test/fold0\car_horn6.jpg
copy succes : 3_image/fold0\car_horn60.jpg -> 4_dataset/3_test/fold0\car_horn60.jpg
copy succes : 3_image/fold0\car_horn61.jpg -> 4_dataset/3_test/fold0\car_horn61.jpg
copy succes : 3_image/fold0\car_horn62.jpg -> 4_dataset/3_test/fold0\car_horn6

...

copy succes : 3_image/fold9\street_music647.jpg -> 4_dataset/2_val/fold9\street_music647.jpg
copy succes : 3_image/fold9\street_music648.jpg -> 4_dataset/2_val/fold9\street_music648.jpg
copy succes : 3_image/fold9\street_music649.jpg -> 4_dataset/2_val/fold9\street_music649.jpg
copy succes : 3_image/fold9\street_music65.jpg -> 4_dataset/2_val/fold9\street_music65.jpg
copy succes : 3_image/fold9\street_music650.jpg -> 4_dataset/2_val/fold9\street_music650.jpg
copy succes : 3_image/fold9\street_music651.jpg -> 4_dataset/2_val/fold9\street_music651.jpg
copy succes : 3_image/fold9\street_music652.jpg -> 4_dataset/2_val/fold9\street_music652.jpg
copy succes : 3_image/fold9\street_music653.jpg -> 4_dataset/2_val/fold9\street_music653.jpg
copy succes : 3_image/fold9\street_music654.jpg -> 4_dataset/2_val/fold9\street_music654.jpg
copy succes : 3_image/fold9\street_music655.jpg -> 4_dataset/2_val/fold9\street_music655.jpg
copy succes : 3_image/fold9\street_music656.jpg -> 4_dataset/2_val/fold9

copy succes : 3_image/fold9\street_music726.jpg -> 4_dataset/2_val/fold9\street_music726.jpg
copy succes : 3_image/fold9\street_music727.jpg -> 4_dataset/2_val/fold9\street_music727.jpg
copy succes : 3_image/fold9\street_music728.jpg -> 4_dataset/2_val/fold9\street_music728.jpg
copy succes : 3_image/fold9\street_music729.jpg -> 4_dataset/2_val/fold9\street_music729.jpg
copy succes : 3_image/fold9\street_music73.jpg -> 4_dataset/2_val/fold9\street_music73.jpg
copy succes : 3_image/fold9\street_music730.jpg -> 4_dataset/2_val/fold9\street_music730.jpg
copy succes : 3_image/fold9\street_music731.jpg -> 4_dataset/2_val/fold9\street_music731.jpg
copy succes : 3_image/fold9\street_music732.jpg -> 4_dataset/2_val/fold9\street_music732.jpg
copy succes : 3_image/fold9\street_music733.jpg -> 4_dataset/2_val/fold9\street_music733.jpg
copy succes : 3_image/fold9\street_music734.jpg -> 4_dataset/2_val/fold9\street_music734.jpg
copy succes : 3_image/fold9\street_music735.jpg -> 4_dataset/2_val/fold9

copy succes : 3_image/fold9\street_music805.jpg -> 4_dataset/2_val/fold9\street_music805.jpg
copy succes : 3_image/fold9\street_music806.jpg -> 4_dataset/2_val/fold9\street_music806.jpg
copy succes : 3_image/fold9\street_music807.jpg -> 4_dataset/2_val/fold9\street_music807.jpg
copy succes : 3_image/fold9\street_music808.jpg -> 4_dataset/2_val/fold9\street_music808.jpg
copy succes : 3_image/fold9\street_music809.jpg -> 4_dataset/2_val/fold9\street_music809.jpg
copy succes : 3_image/fold9\street_music81.jpg -> 4_dataset/2_val/fold9\street_music81.jpg
copy succes : 3_image/fold9\street_music810.jpg -> 4_dataset/2_val/fold9\street_music810.jpg
copy succes : 3_image/fold9\street_music811.jpg -> 4_dataset/2_val/fold9\street_music811.jpg
copy succes : 3_image/fold9\street_music812.jpg -> 4_dataset/2_val/fold9\street_music812.jpg
copy succes : 3_image/fold9\street_music813.jpg -> 4_dataset/2_val/fold9\street_music813.jpg
copy succes : 3_image/fold9\street_music814.jpg -> 4_dataset/2_val/fold9

copy succes : 3_image/fold9\street_music884.jpg -> 4_dataset/3_test/fold9\street_music884.jpg
copy succes : 3_image/fold9\street_music885.jpg -> 4_dataset/3_test/fold9\street_music885.jpg
copy succes : 3_image/fold9\street_music886.jpg -> 4_dataset/3_test/fold9\street_music886.jpg
copy succes : 3_image/fold9\street_music887.jpg -> 4_dataset/3_test/fold9\street_music887.jpg
copy succes : 3_image/fold9\street_music888.jpg -> 4_dataset/3_test/fold9\street_music888.jpg
copy succes : 3_image/fold9\street_music889.jpg -> 4_dataset/3_test/fold9\street_music889.jpg
copy succes : 3_image/fold9\street_music89.jpg -> 4_dataset/3_test/fold9\street_music89.jpg
copy succes : 3_image/fold9\street_music890.jpg -> 4_dataset/3_test/fold9\street_music890.jpg
copy succes : 3_image/fold9\street_music891.jpg -> 4_dataset/3_test/fold9\street_music891.jpg
copy succes : 3_image/fold9\street_music892.jpg -> 4_dataset/3_test/fold9\street_music892.jpg
copy succes : 3_image/fold9\street_music893.jpg -> 4_dataset/3

copy succes : 3_image/fold9\street_music962.jpg -> 4_dataset/3_test/fold9\street_music962.jpg
copy succes : 3_image/fold9\street_music963.jpg -> 4_dataset/3_test/fold9\street_music963.jpg
copy succes : 3_image/fold9\street_music964.jpg -> 4_dataset/3_test/fold9\street_music964.jpg
copy succes : 3_image/fold9\street_music965.jpg -> 4_dataset/3_test/fold9\street_music965.jpg
copy succes : 3_image/fold9\street_music966.jpg -> 4_dataset/3_test/fold9\street_music966.jpg
copy succes : 3_image/fold9\street_music967.jpg -> 4_dataset/3_test/fold9\street_music967.jpg
copy succes : 3_image/fold9\street_music968.jpg -> 4_dataset/3_test/fold9\street_music968.jpg
copy succes : 3_image/fold9\street_music969.jpg -> 4_dataset/3_test/fold9\street_music969.jpg
copy succes : 3_image/fold9\street_music97.jpg -> 4_dataset/3_test/fold9\street_music97.jpg
copy succes : 3_image/fold9\street_music970.jpg -> 4_dataset/3_test/fold9\street_music970.jpg
copy succes : 3_image/fold9\street_music971.jpg -> 4_dataset/3

In [7]:
import glob

print("class 별 data set 개수 측정")
print("========== car_horn ==========")
print('train car_horn image count : ' + str(len(glob.glob("4_dataset/1_train/fold0/*jpg"))))
print('validation car_horn image count : ' + str(len(glob.glob("4_dataset/2_val/fold0/*jpg"))))
print('test car_horn image count : ' + str(len(glob.glob("4_dataset/3_test/fold0/*jpg"))))

print("========== dog_bark ==========")
print('train dog_bark image count : ' + str(len(glob.glob("4_dataset/1_train/fold1/*jpg"))))
print('validation dog_bark image count : ' + str(len(glob.glob("4_dataset/2_val/fold1/*jpg"))))
print('test dog_bark image count : ' + str(len(glob.glob("4_dataset/3_test/fold1/*jpg"))))

print("========== drilling ==========")
print('train drilling image count : ' + str(len(glob.glob("4_dataset/1_train/fold2/*jpg"))))
print('validation drilling image count : ' + str(len(glob.glob("4_dataset/2_val/fold2/*jpg"))))
print('test drilling image count : ' + str(len(glob.glob("4_dataset/3_test/fold2/*jpg"))))

print("========== gun_shot ==========")
print('train gun_shot image count : ' + str(len(glob.glob("4_dataset/1_train/fold3/*jpg"))))
print('validation gun_shot image count : ' + str(len(glob.glob("4_dataset/2_val/fold3/*jpg"))))
print('test gun_shot image count : ' + str(len(glob.glob("4_dataset/3_test/fold3/*jpg"))))

print("========== siren ==========")
print('train siren image count : ' + str(len(glob.glob("4_dataset/1_train/fold4/*jpg"))))
print('validation siren image count : ' + str(len(glob.glob("4_dataset/2_val/fold4/*jpg"))))
print('test siren image count : ' + str(len(glob.glob("4_dataset/3_test/fold4/*jpg"))))

print("========== 배경음 실험 데이터1 ==========")
print('train background1 image count : ' + str(len(glob.glob("4_dataset/1_train/fold5/*jpg"))))
print('validation background1 image count : ' + str(len(glob.glob("4_dataset/2_val/fold5/*jpg"))))
print('test background1 image count : ' + str(len(glob.glob("4_dataset/3_test/fold5/*jpg"))))

print("========== 배경음 실험 데이터2 ==========")
print('train background2 image count : ' + str(len(glob.glob("4_dataset/1_train/fold6/*jpg"))))
print('validation background2 image count : ' + str(len(glob.glob("4_dataset/2_val/fold6/*jpg"))))
print('test background2 image count : ' + str(len(glob.glob("4_dataset/3_test/fold6/*jpg"))))

print("========== 배경음 실험 데이터3 ==========")
print('train background13 image count : ' + str(len(glob.glob("4_dataset/1_train/fold7/*jpg"))))
print('validation background3 image count : ' + str(len(glob.glob("4_dataset/2_val/fold7/*jpg"))))
print('test background3 image count : ' + str(len(glob.glob("4_dataset/3_test/fold7/*jpg"))))

print("========== 배경음 실험 데이터4 ==========")
print('train background4 image count : ' + str(len(glob.glob("4_dataset/1_train/fold8/*jpg"))))
print('validation background4 image count : ' + str(len(glob.glob("4_dataset/2_val/fold8/*jpg"))))
print('test background4 image count : ' + str(len(glob.glob("4_dataset/3_test/fold8/*jpg"))))

print("========== 배경음 실험 데이터4 ==========")
print('train background5 image count : ' + str(len(glob.glob("4_dataset/1_train/fold9/*jpg"))))
print('validation background5 image count : ' + str(len(glob.glob("4_dataset/2_val/fold9/*jpg"))))
print('test background5 image count : ' + str(len(glob.glob("4_dataset/3_test/fold9/*jpg"))))


class 별 data set 개수 측정
========== car_horn ==========
train car_horn image count : 258
validation car_horn image count : 85
test car_horn image count : 86
========== dog_bark ==========
train dog_bark image count : 407
validation dog_bark image count : 135
test dog_bark image count : 136
========== drilling ==========
train drilling image count : 487
validation drilling image count : 162
test drilling image count : 161
========== gun_shot ==========
train gun_shot image count : 95
validation gun_shot image count : 31
test gun_shot image count : 31
========== siren ==========
train siren image count : 539
validation siren image count : 179
test siren image count : 179
========== 배경음 실험 데이터1 ==========
train background1 image count : 599
validation background1 image count : 199
test background1 image count : 199
========== 배경음 실험 데이터2 ==========
train background2 image count : 586
validation background2 image count : 195
test background2 image count : 195
========== 배경음 실험 데이터3 =========